In [1]:
import cv2
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from PIL import Image
import os
import shutil
import numpy as np
import pandas as pd
import random
import glob
import csv

In [15]:
# Define this path as you like
# This probably would change according to the folder where you are going to save the data
destination_folder = '../data/CNR-Park_EXT_yolov8-2/train/'  

# Data paths
source_dataset_path = '../data/CNR-Park_EXT_yolov8-2/train/images'
annotation_path = '../data/CNR-Park_EXT_yolov8-2/train/labels'


# Read and get the files as a list from the .csv
#csv_file_path = '../data/image_list.csv'    #=>  change this to the folder that consists the image_list

In [16]:
jpg_list = os.listdir(source_dataset_path)

In [17]:
jpg_list[0][:-4]

'2015-11-16_0714_C02_jpg.rf.cc0822a4c0a0df4280fad621e10ee368'

In [18]:
# Make the two folders inside datasets/images and datasets/labels_xml
images_path = destination_folder + '/' + 'images'
labels_txt_path = destination_folder + '/' + 'labels'
#labels_xml_path = destination_folder+ '/' + 'labels_xml'


#os.makedirs(images_path + '/ + 'images', exist_ok=True)
#os.makedirs(labels_xml_path, exist_ok=True)
#os.makedirs(labels_txt_path , exist_ok=True)

# Move images to 'images' folder
for filename in os.listdir(destination_folder):
    if filename.endswith(".jpg"):  # Assuming your images have a specific extension, like .jpg
        src_path = os.path.join(destination_folder, filename)
        dest_path = os.path.join(images_path, filename)
        shutil.move(src_path, dest_path)

# Move labels to 'labels_xml' folder
''' 
for filename in os.listdir(destination_folder):
    if filename.endswith(".xml"):  # Assuming your labels have a specific extension, like .xml
        src_path = os.path.join(destination_folder, filename)
        dest_path = os.path.join(labels_xml_path, filename)
        shutil.move(src_path, dest_path)
'''

## Train, val, test split 

In [19]:
# Paths and settings
train_path = destination_folder+ '/' + 'train'
val_path = destination_folder+ '/' + 'val'
test_path = destination_folder+ '/' + 'test'

os.makedirs(train_path + '/' + 'images', exist_ok=True)
os.makedirs(train_path + '/' + 'labels', exist_ok=True)
os.makedirs(val_path + '/' + 'images', exist_ok=True)
os.makedirs(val_path + '/' + 'labels', exist_ok=True)
os.makedirs(test_path + '/' +'images', exist_ok=True)
os.makedirs(test_path + '/' + 'labels', exist_ok=True)

In [20]:
def save_files(files, im_source, destination_images, destination_labels):
    for file_path in files:
        # Extract file name without extension
        file_name = os.path.splitext(os.path.basename(file_path))[0]

        # Define paths for images and labels
        image_source_path = os.path.join(im_source, f"{file_name}.jpg")  # Assuming images have the same name as the labels with a different extension
        label_source_path = file_path

        image_destination_path = os.path.join(destination_images, f"{file_name}.jpg")
        label_destination_path = os.path.join(destination_labels, f"{file_name}.txt")

        # Copy or move image and label files to the destination
        shutil.copy(image_source_path, image_destination_path)  # Change to shutil.move if you want to move instead of copy
        shutil.copy(label_source_path, label_destination_path)

In [21]:
# Get all txt files and split them
txt_files = glob.glob(labels_txt_path + '/' + '*.txt')
random.shuffle(txt_files)

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

train_split = int(train_ratio * len(txt_files))
val_split = int((train_ratio + val_ratio) * len(txt_files))

train_files = txt_files[:train_split]
val_files = txt_files[train_split:val_split]
test_files = txt_files[val_split:]

# Process training, validation, and test files (same as before, but now includes selected_class)
save_files(train_files, images_path, train_path +'/'+ 'images', train_path +'/'+ 'labels')
save_files(val_files, images_path, val_path +'/'+ 'images', val_path +'/'+  'labels')
save_files(test_files, images_path, test_path +'/'+ 'images', test_path +'/'+  'labels')
